## Load common code

In [ ]:
#For testing, leave commented
#swivel = 1
#swivel_order = [2, 1, 22, 7, 16, 6, 3, 12, 10, 9, 4, 5, 15, 8, 14, 0, 20, 13, 19, 11, 18, 21, 17]#
#
#desired_file_prefix = "F_GENDER" 
#
#normalize_data=True
#
#try:
#    if (len(desired_file_output) == 0):
#        desired_file_output = ""
#except:
#    desired_file_output = "" 

In [ ]:
#For testing
#desired_file_prefix = "NF_GENDER" 
#normalize_data=True
#desired_file_output = ""

%run ./common-code.ipynb

## Select Generic Database to work with

In [ ]:
genericDB_files_dict = {}

#desired_file_prefix = "F_GENDER" 

## Create above database dictionary

In [ ]:
import os

filelist = os.listdir('../DATA')

for file in filelist:
    if (file.find(desired_file_prefix)==0):
        print(file)
        fname = "../DATA/"+file
        afile = open(fname, "r")
        data = afile.read()
        name = file.split("_all")[0]
        genericDB_files_dict.setdefault(name, []).append(data)
        #print(data)
        afile.close()

In [ ]:
#print(genericDB_files_dict)
print(genericDB_files_dict.keys())

# Create a dictionary of soc_codes
#
genericDB_dict = {}

for db in genericDB_files_dict.keys():
    
    genericDB = genericDB_files_dict[db]
    
    #print(genericDB[0])    
    line = genericDB[0].replace("\"","")
    genericDB_array= line.split("\n")
    
    #print(line)
    #print("*********")
    #print(genericDB_array)

## Curve fit - nonlinear

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [1]:
def func(x, a, b, c):
    return a * np.exp(b * x) + c

# Analysis

In [ ]:
# Create a dictionary of soc_codes
#
genericDB_dict = {}


# Create Dictionary of SOC codes
#
for db in genericDB_files_dict.keys():
    
    genericDB = genericDB_files_dict[db]
    
    #print(genericDB[0])    
    line = genericDB[0].replace("\"","")
    genericDB_array= line.split("\n")
    #print(genericDB_array[12:])
    
    # Line 12 is where the header stops and the data starts
    #
    for line in genericDB_array[12:]:
        line_array = line.split(",")
        #key  = line_array[0].    [0:7].replace("-","")
        key  = line_array[0][6:12]
        data = line_array[1:]
        #print(line_array)
        #print(key,data)

        # Replace empty values with zero
        if (len(key) > 1):
            #print(key,data[0:8])
            for index in range(0,8):
                if (len(data[index]) == 0):
                    data[index] = '0'
            #print(key,data[0:8])
        
        # Convert to Integers
        if (len(key) > 1):
            #print(key,data[0:8])
            for index in range(0,8):
                data[index] = int(data[index])
            #print(key,data[0:8])   
            
        # Create dictionary entry
        if (len(key) > 1):
            genericDB_dict.setdefault(db+"_"+key, []).append(data[0:8])
    
#index = 0
#for key in genericDB_dict:
#    print(index, key, genericDB_dict[key])
#    index = index + 1

In [ ]:
#print(genericDB_dict)

## Scatter plot level 1 RELATIVE average count of DATABASE data for soc groups

In [ ]:
%matplotlib inline

In [ ]:
#print(genericDB_dict.keys())

In [ ]:
# Create list of new level 1 codes

soc_new_lv1 = []

for key in soc_new_code:
    
    # Grab first two chars of code
    lvindex = key[0][0:2]
    
    #Eliminate "00" ALL
    #if (lvindex.find("00") < 0):
    
    if lvindex not in soc_new_lv1:
        soc_new_lv1.append(lvindex)
        #print(lvindex)

print(soc_new_lv1)

In [ ]:
def find_order(data):
    
    atmp = []
    for i in range(0,len(data)):
        atmp.append( (i,data[i]) )
        
    #print("HERE")
    #print(atmp)
    
    atmp2 = sorted(atmp, key = lambda x: x[1])

    #print("THERE")
    #print(atmp2)
    
    l = []
    for i in range(0,len(atmp2)):
        l.append(atmp2[i][0])
        
    return(l)

In [ ]:
no_genericDB_data_dict = {}

# running generic_totals for 2011 to 2018 
for db in genericDB_files_dict.keys():
    x = []
    y = []
    count = 0 
    zero_count = 0
        
    #for index in level1_sockeys.keys():
    for index in soc_new_lv1:
        #print(index,db)
    
        if (index.find("00") == 0):
            if (desired_file_prefix[0:3] == "NF_"):
                new_key = db+"_"+"0XXXXX"
            else:
                new_key = db+"_"+index+"0000"
        else:
            if (desired_file_prefix[0:3] == "NF_"):
                new_key = db+"_"+index+"0000"
            else:
                new_key = db+"_"+index+"XXXX"
            
        #print(new_key)
    
        if (len(index) > 0):
            generic_totals = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
            
            for db_key in genericDB_dict.keys():
                #print(db_key,new_key)
                if (db_key.find(new_key) == 0):
                    #print(db_key,new_key)
                    #print("found "+db_key)
                    #print(genericDB_dict[db_key][0])
                    generic_totals = generic_totals + np.array(genericDB_dict[db_key][0])
                    #print("*",generic_totals)             
                 
            #print(generic_totals)   
            #relative_totals = fatal_totals / generic_totals
            relative_totals = generic_totals
            
            print(db+" - "+index,relative_totals)
            
            count=count+1
            #value = -1*math.log10(np.mean(relative_totals))
            value = np.mean(relative_totals)
            
            x.append(count-1)
            y.append(value)
            
            if (value <= no_data_threshold):
                zero_count = zero_count + 1
                
    # Eliminate the first value
    y = y[1:]
    x = x[1:]
                
    if (normalize_data==True):            
        # Normalize to the total value
        #total_value = np.sum(y[1:])
        #y = y / total_value
        if (db.find("_GENDER")>=0 and db.find("_F")>=0):
            y = np.array(y) / (np.array(total_female)*1000)
            
        elif (db.find("_GENDER")>=0 and db.find("_M")>=0):
            y = np.array(y) / (np.array(total_male)*1000)
            
        elif (db.find("_R")>=0 and db.find("_White")>=0):
            y = np.array(y) / (np.array(total_white)*1000)
            
        elif (db.find("_R")>=0 and db.find("_Black")>=0):
            y = np.array(y) / (np.array(total_black)*1000)
            
        elif (db.find("_R")>=0 and db.find("_Asian")>=0):
            y = np.array(y) / (np.array(total_asian)*1000)
            
        elif (db.find("_R")>=0 and db.find("_Hispanic")>=0):
            y = np.array(y) / (np.array(total_asian)*1000)
            
        else:
            total_value = np.sum(y[1:])
            y = np.array(y) / (np.array(total_total)*1000) 

                
    #Append data set
    global_y_soc1.append([db,y])
    
    # Check for empty sets
    if (zero_count > 10):
        if (db not in global_soc1_empty):
            global_soc1_empty.append(db) 
         
    print(db,y)
    print(" ")
    
    if (swivel == 1):
        # What is the best swivel order
        print("Suggested swivel order")
        print(find_order(y))
        
        # Swivel categories
        #if (db.find("NF_")>=0):
        #    swivel_order = [22, 2, 3, 6, 1, 8, 4, 0, 16, 15, 7, 14, 5, 9, 18, 12, 20, 17, 13, 19, 10, 21, 11]
        #else:
        #    swivel_order =  [2, 1, 22, 7, 16, 6, 3, 12, 10, 9, 4, 5, 15, 8, 14, 0, 20, 13, 19, 11, 18, 21, 17]
        #swivel_order =  [2, 1, 22, 7, 16, 6, 3, 12, 10, 9, 4, 5, 15, 8, 14, 0, 20, 13, 19, 11, 18, 21, 17]
       
        tmpy = y.copy()
        for i in range(0,len(swivel_order)):
            y[i] = tmpy[swivel_order[i]] 

    print(db,y)
    print(" ")
    
    
    # Curve fit
    r = []
    try:
        popt, pcov = curve_fit(func, x, y)
    except:
        popt = [0,0,0]
    
    popt
    for i in range(0,len(x)):
        r.append(func(i, *popt))
    
 
    title = desired_file_output + str(db) + ", mean per SOC code level 1"
    
    plt_title = title + ", " + str(popt)
    plt.figure(title, figsize=(10, 10))
    #plt.figure(title)
    plt.title(plt_title)
    plt.xlabel('categories')
    plt.ylabel('probability index')
    
    # ----------------------------------
    
    #plt.plot(x,y,'o')
    
    if (swivel == 1):
        job_code = soc_swivel_code
    else:
        job_code = soc_new_code
    
    total_len = len(job_code) - 1
    
    #for index in range(0,total_len):  
    #    #plabel = str(index) + " " + str(soc_labels[index])
    #    plabel = str(index) + " " + str(job_code[index+1][1])
    #    plt.plot(x[index],y[index],'-',label=plabel, color="black")
    #    plt.text(x[index],y[index],str(index))  
        
    for index in range(0,total_len):  
        plabel = str(index) + " " + str(job_code[index+1][1])
        plt.plot(x[index],y[index],'o',label=plabel, markersize=4, color='black')
        #plt.plot(x[index], func(index, *popt) ,'or', markersize=2)
        plt.text(x[index],y[index],str(index))
        #print(x[index], func(index, *popt))
        
    # Plot line regression
    plt.plot(x, r ,'-g')
        
        
    plt.legend(loc="upper left", ncol= 1, fontsize="small")   
    
    # ----------------------------------
    
    save_title = "../PLOTS/" + title + ".png"
    save_title = save_title.replace(",","")
    save_title = save_title.replace(" ","_")
    plt.savefig(save_title, dpi=100, bbox_inches='tight')
    

## Functions

In [1]:
def plot_avg(name,data):
    global swivel
    
    # Creat plot data
    x = data[0]
    y = data[1]
    count = 0
    
    # Find best swivel order
    print("Best swivel order for combined data")
    print(find_order(y))
    
    # Curve fit
    r = []
    try:
        popt, pcov = curve_fit(func, x, y)
    except:
        popt = [0,0,0]
    
    popt
    for i in range(0,len(x)):
        r.append(func(i, *popt))
        
    # Plot the data
    title = "SOC level 1 "+name+" combined variables averaged index"
    
    plt_title = title + ", " + str(popt) 

    plt.figure(title, figsize=(10, 10))
    plt.title(plt_title)
    plt.xlabel('categories')
    plt.ylabel('probability index')
    
    max_value = max(y)
    #plt.ylim(0,12)
    #plt.ylim(0,max_value*1.2)
    
    if (swivel == 1):
        job_code = soc_swivel_code
    else:
        job_code = soc_new_code
    
    
    total_len = len(job_code) - 1
    
    # Plot points
    for index in range(0,total_len):  
        plabel = str(index) + " " + str(job_code[index+1][1])
        plt.plot(x[index],y[index],'o',label=plabel, markersize=4, color='black')
        #plt.plot(x[index], func(index, *popt) ,'or', markersize=2)
        plt.text(x[index],y[index],str(index))
        #print(x[index], func(index, *popt))
        
    # Plot line regression
    plt.plot(x, r ,'-g')

    #plt.legend(loc="upper left", bbox_to_anchor=(0.72, 0.3), ncol= 1, fontsize="small")
    plt.legend(loc="upper left", ncol= 1, fontsize="small")
    
    save_title = "../PLOTS/" + desired_file_output + title + ".png"
    save_title = save_title.replace(",","")
    save_title = save_title.replace(" ","_")
    plt.savefig(save_title, dpi=100, bbox_inches='tight')
    
    plt.show()
    plt.close()

In [ ]:
def color_corr_green(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'green' if val > 0.8 else 'black'
    return 'color: %s' % color

In [ ]:
def avg_data(list_to_avg):
    
    length = len(global_y_soc1[0][1])
    
    # Create plot data
    x = np.zeros(length)
    count = 0.0

    #print(global_y_soc1)
    
    # Init to first found array ( (5 + 5) / 2 = 10 , no need to worry about first sample repeat )
    y = np.zeros(length)
    y = y + global_y_soc1[0][1]
    
    for to_avg in avg_list:
        for index in range(0,len(global_y_soc1)):
            #print(index,to_avg)
            cat = str(global_y_soc1[index][0])
            print(cat,to_avg)
            if (cat.find(to_avg)>=0):
                print(global_y_soc1[index][0])
                print(global_y_soc1[index][1])
                y = (y + global_y_soc1[index][1]) / 2
    
    for index in range(0,23):
        x[index] = count
        count=count+1.0
        
    data = np.array([x,y])

    return(data)

## Analysis Box Plots

In [ ]:
print(global_y_soc1)

In [ ]:
avg_list = []

filelist = os.listdir('../DATA')
for file in filelist:
    if (file.find(desired_file_prefix)==0):
        fname = file.split("_all")[0]
        #print(fname)|
        avg_list.append(fname)
        
print(avg_list)

#avg_list = ['F_AGE_14to15', 'F_AGE_16to19',  'F_AGE_20to24', 'F_AGE_25to34', 'F_AGE_35to44', 'F_AGE_45to54', 'F_AGE_55to64', 'F_AGE_65plus']

data = avg_data(avg_list)

In [ ]:
plot_avg(desired_file_prefix,data)

In [ ]:
print(len(global_y_soc1))
print(global_y_soc1)

In [ ]:
# Creating pandas |dataframe from numpy array
#pd_data_df = pd.DataFrame({ global_y_soc1[0][0] : global_y_soc1[0][1] })
pd_df = pd.DataFrame()

for index in range(0,len(global_y_soc1)):
    pd_df[global_y_soc1[index][0]] = global_y_soc1[index][1]

print(pd_df)

title = desired_file_prefix + " boxplots"
plt.figure(title,figsize=(10, 10))
plt.title(title)

#plt.ylim(0.0, 0.15)

pd_df.boxplot()
plt.xticks(rotation = 90)
plt.tight_layout()

save_title = "../PLOTS/" + desired_file_output + title + ".png"
save_title = save_title.replace(",","")
save_title = save_title.replace(" ","_")
plt.savefig(save_title, dpi=100, bbox_inches='tight')